In [12]:
# Project Name: Next-Out
# Description: Summarize data from many files into a single Dataframe or Excel file. Export out a new "Next-Out" file (*.no) to reduce post-processing time. 
# Copyright (c) 2024 Justin Edenbaum, Never Gray
#
# This file is licensed under the MIT License.
# You may obtain a copy of the license at https://opensource.org/licenses/MIT

from pathlib import Path
import NO_parser
import pandas as pd
import gzip
import pickle

# This function creates a Next-Out file, which is a compressed pickle


In [ ]:
def create_no_file(output_file_path):
    # Parse the file
    d, output_meta_data = NO_parser.parse_file(output_file_path, gui="", conversion_setting="SI")
    
    # Define the output file path with the .no suffix
    no_file_path = output_file_path.with_suffix('.no')
    
    # Save the variable 'd' to the file with compression
    with gzip.open(no_file_path, 'wb') as file:
        no_file = {'output_file_path': output_file_path, 'd': d, 'output_meta_data': output_meta_data}
        pickle.dump(no_file, file)
    
    print(f"{no_file_path} created.")

In [14]:
def get_fire_location_and_airflow(no_file_path):
    # Load the variable 'd' and 'output_meta_data' from the pickled file
    with gzip.open(no_file_path, 'rb') as file:
        no_file = pickle.load(file)
        d = no_file['d']
        output_meta_data = no_file['output_meta_data']

    output_meta_data = no_file['output_meta_data']
    last_time_value = d['SSA'].index.get_level_values('Time').max()
    fire_segment = int(output_meta_data['form4_df'].index.get_level_values('Segment')[0])
    airflow_value = d['SSA'].loc[(last_time_value, fire_segment), 'Airflow']

    return fire_segment, airflow_value

In [15]:
def get_airflow(no_file_path, segment):
    # Load the variable 'd' and 'output_meta_data' from the pickled file
    with gzip.open(no_file_path, 'rb') as file:
        no_file = pickle.load(file)
        d = no_file['d']
        output_meta_data = no_file['output_meta_data']

    output_meta_data = no_file['output_meta_data']
    last_time_value = d['SSA'].index.get_level_values('Time').max()
    airflow_value = d['SSA'].loc[(last_time_value, segment), 'Airflow']

    return airflow_value

In [16]:
# Create NO Files for all SES output files in a directory
def create_no_files(directory_path):
    # Iterate over all files with a suffix of '.out' in the directory
    for output_file_path in directory_path.glob('*.out'):
        create_no_file(output_file_path)

In [17]:
#Summarize all airflows at the fire site. Only works for files with a fire. 
def summarize_fire_airflow(no_directory_path):
    summary_data = []

    # Iterate over all files with a suffix of '.no' in the directory
    for file_path in no_directory_path.glob('*.no'):
        try:
            # Get the fire segment and airflow value
            fire_segment, airflow_value = get_fire_location_and_airflow(file_path)
            
            # Append the data to the summary list
            summary_data.append({
                'File Name': file_path.stem,
                'Fire Segment': fire_segment,
                'Airflow': abs(airflow_value)
            })
        except:
            print(f"Error processing {file_path}")

    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data)

    # Display the summary DataFrame
    print(summary_df)
    # Save the summary DataFrame as an Excel file in the same directory
    summary_df.to_excel(no_directory_path / 'fire_summary.xlsx', index=False)


In [18]:
#Summarizes airflow for multiple segment lookups at different segments for all NO Files in a directory
def summarize_airflow_4_segments(no_directory_path, segment_lookup):
    summary_data = []

    # Iterate over all files with a suffix of '.no' in the directory
    for no_file_path in directory_path.glob('*.no'):
        # Get the fire segment and airflow value
        print(no_file_path)
        for key in segment_lookup:
            airflow_value = get_airflow(no_file_path, segment_lookup[key])
            if airflow_value is not None:
                # Append the data to the summary list
                summary_data.append({
                    'File Name': no_file_path.stem,
                    'Segment': segment_lookup[key],
                    'Airflow': (airflow_value)
                })
        
    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data)

    # Pivot the DataFrame so that each segment becomes a column
    pivot_df = summary_df.pivot(index='File Name', columns='Segment', values='Airflow').reset_index()

    # Display the pivoted DataFrame
    print(pivot_df)

    # Save the pivoted DataFrame as an Excel file in the same directory
    pivot_df.to_excel(directory_path / 'pivot_segment_summary.xlsx', index=False)

In [ ]:
segment_lookup={
    "500":500,
    "532":532,
    "534":534,
    "536":536,
    "538":538,
}
directory_path = Path('C:\\Simulations\\cosburn')
#create_no_files(directory_path)
summarize_airflow_4_segments(directory_path, segment_lookup)

Run msg: Importing data from PT05-E1GM005-100WG.OUT.
Run msg: Finished importing data from PT05-E1GM005-100WG.OUT.
C:\Simulations\cosburn\PT05-E1GM005-100WG.no created.
Run msg: Importing data from PT05-E1GM005-101WG.OUT.
Run msg: Finished importing data from PT05-E1GM005-101WG.OUT.
C:\Simulations\cosburn\PT05-E1GM005-101WG.no created.
Run msg: Importing data from PT05-E1GM006-100WG.OUT.
Run msg: Finished importing data from PT05-E1GM006-100WG.OUT.
C:\Simulations\cosburn\PT05-E1GM006-100WG.no created.
Run msg: Importing data from PT05-E1GM006-101WG.OUT.
Run msg: Finished importing data from PT05-E1GM006-101WG.OUT.
C:\Simulations\cosburn\PT05-E1GM006-101WG.no created.
Run msg: Importing data from PT05-E1GM006-101WM.OUT.
Run msg: Finished importing data from PT05-E1GM006-101WM.OUT.
C:\Simulations\cosburn\PT05-E1GM006-101WM.no created.
Run msg: Importing data from PT05-E1GM007-100WG.OUT.
Run msg: Finished importing data from PT05-E1GM007-100WG.OUT.
C:\Simulations\cosburn\PT05-E1GM007-100W